<div>
<img src="../image/images.png" width="100%"/>
</div>

<h1>
    <center>
        H SAF H64 Accumulated Precipitation Product
        <br>
        <br>
        Data Download
    </center>
</h1>

This section guides you through downloading the H64 data product using the `download_h64()` method from the `H64DataProcessor` library. The process is designed to be user-friendly, leveraging widgets for interactive input.  

### Key Steps  

1. **User Authentication**:  
   Use your H-SAF credentials (username and password) to access the data server securely.  

2. **Specify the Time Period**:  
   Set the start and end dates for the desired data range using the date picker widgets.  

3. **Define the Area of Interest (AOI)**:  
   You can define the study area in two ways:  
   - **Bounding Box**: Provide corner coordinates (lower left, lower right, upper right, and upper left) in WGS 84 format.  
   - **Shapefile**: Upload a shapefile containing the AOI. Ensure the file is in the correct format and includes accurate spatial information.  

4. **Download Process**:  
   Based on your AOI selection (bounding box or shapefile), the `download_h64()` method retrieves data for the specified time period and region. The downloaded data is stored in a `data` folder for easy access and organization.  

### Error Handling  
If any issues arise during the download process (e.g., invalid credentials, unsupported AOI format), appropriate error messages will be displayed to guide you in resolving them.

By following these steps, you can ensure seamless access to the H64 dataset, enabling further analysis and visualization in subsequent sections.

### Libraries import
This code cell imports the necessary libraries and modules required for the download and processing of the H64 Accumulated Precipitation data product. These libraries facilitate various operations, such as date handling, user interface creation, geospatial data manipulation, and the custom functionality for interacting with the H64 data server.

In [ ]:
import os
from datetime import date

import numpy as np
import ipywidgets as widgets
import geopandas as gpd
from shapely.geometry import Polygon

from hsaf_data_access import HSAFDataAccess as data_access

### User Authentication  
This code cell creates widgets for entering your H-SAF username and password. These credentials are required for accessing and downloading H64 data from the H-SAF FTP server.  
Ensure you have valid credentials. If you don’t, you can register on the [H-SAF website](https://hsaf.meteoam.it/).


In [ ]:
# User authentication widgets
username = widgets.Text(description='Username:', disabled=False)
psw = widgets.Password(description
                       ='Password:', disabled=False)



display(widgets.HBox([username, psw]))

### Specify Time Period  
This cell initializes date picker widgets for specifying the start and end dates of the desired data range.  
- The selected dates define the period for which H64 data will be downloaded.  
- Ensure the end date is not earlier than the start date.

In [ ]:
# Date selection widgets
datestart = widgets.DatePicker(description='Start Date', disabled=False, value=date.today(), max=date.today())
dateend = widgets.DatePicker(description='End Date', disabled=False, value=date.today(), max=date.today())
display(widgets.HBox([datestart, dateend]))

### Define Area of Interest (AOI)  
This cell provides two options for defining the AOI:  
1. **Bounding Box**: Use numeric inputs to specify the coordinates of the region of interest. The format must align with the WGS 84 coordinate system.  
2. **Shapefile**: Upload a shapefile that outlines your region of interest. Ensure the shapefile is accurate and in the required format.  
A tab widget allows you to switch between the bounding box and shapefile options.



In [ ]:
aoi_shp = data_access.create_shp_widgets()
aoi_box = data_access.create_box_widgets()
aoi_tab = widgets.Tab()
tab_contents = ['Area by Bounding Box', 'Area by Shapefile']
children = [aoi_box, aoi_shp]
aoi_tab.children = children
for i in range(len(tab_contents)):
    aoi_tab.set_title(i, tab_contents[i])
aoi_tab

### Data Download  
This cell executes the `download_h64()` method based on the user inputs:  
- The AOI and date range are passed to the method.  
- If using a bounding box, the coordinates are converted into a polygon for processing.  
- If using a shapefile, it is reprojected into the required coordinate reference system.  

The cell also includes error handling to notify the user of any issues (e.g., incorrect credentials, invalid AOI format, or server issues).

In [ ]:
try:
    # Get the value of the AOI object
    if aoi_tab.selected_index==0:
        bb_coord = []
        aoi = (aoi_tab.children[0].children[1:])
        for child in aoi:
            bb_coord.append(child.value)
        x = np.array([bb_coord[0], bb_coord[1], bb_coord[1], bb_coord[0]])
        y = np.array([bb_coord[2], bb_coord[2], bb_coord[3], bb_coord[3]])
        polygon = Polygon(zip(x, y))
        aoi_gdf = gpd.GeoDataFrame(index=[0], crs="EPSG:4326", geometry=[polygon])
        aoi_gdf = aoi_gdf.to_crs("EPSG:4326")
        
        data_access.download_h64(username.value, psw.value, datestart.value, dateend.value, aoi_gdf)

    elif aoi_tab.selected_index==1:
        shp_path = (aoi_tab.children[1].children[0].value)

        # Read the shapefile to start the spatial analysis
        aoi_gdf = gpd.read_file(shp_path)

        # Reproject the shapefile to the CRS of data
        aoi_gdf = aoi_gdf.to_crs("EPSG:4326")

        # Proceed to download the data
        data_access.download_h64(username.value, psw.value, datestart.value, dateend.value, aoi_gdf)

except Exception as e:
    print(f'Error downloading data: {str(e)}')
    

### Notes  
- Ensure all required libraries (e.g., `xarray`, `geopandas`, `shapely`) are installed and properly configured.  
- Verify that the AOI coordinates or shapefile are within the supported geographical range of the H64 dataset.

<table style="width: 100%; border-collapse: collapse;">
    <tr>
        <td style="text-align: left; font-size: 15px;">
            <a href="overview.ipynb" style="text-decoration:none;">&larr; Overview of H64 Product</a>
        </td>
        <td style="text-align: right; font-size: 15px;">
            <a href="data_visualisation.ipynb" style="text-decoration:none;">Data Visualisation &rarr;</a>
        </td>
    </tr>
</table>